## Codio Activity 6.4: Adjusting Parameters for Variance

**Expected Time: 60 Minutes**

**Total Points: 20 Points**

This activity focuses on using the $\Sigma$ matrix to limit the principal components based on how much variance should be kept.  In the last activity, a screen plot was used to see when the difference in variance explained slows. 

Here, you will determine how many components are required to explain a proportion of variance.  The dataset is a larger example of a housing dataset related to individual houses and features in Ames Iowa.  For our purposes the non-null numeric data is selected.

## Index:

- [Problem 1](#Problem-1)
- [Problem 2](#Problem-2)
- [Problem 3](#Problem-3)
- [Problem 4](#Problem-4)

In [2]:
import numpy as np
from scipy.linalg import svd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.datasets import fetch_openml

In [3]:
#fetching the data
housing = fetch_openml(name="house_prices", as_frame=True, data_home='data')

/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [5]:
#examine the dataframe
housing.frame

Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0        1          60       RL         65.0     8450   Pave  None      Reg   
1        2          20       RL         80.0     9600   Pave  None      Reg   
2        3          60       RL         68.0    11250   Pave  None      IR1   
3        4          70       RL         60.0     9550   Pave  None      IR1   
4        5          60       RL         84.0    14260   Pave  None      IR1   
...    ...         ...      ...          ...      ...    ...   ...      ...   
1455  1456          60       RL         62.0     7917   Pave  None      Reg   
1456  1457          20       RL         85.0    13175   Pave  None      Reg   
1457  1458          70       RL         66.0     9042   Pave  None      Reg   
1458  1459          20       RL         68.0     9717   Pave  None      Reg   
1459  1460          20       RL         75.0     9937   Pave  None      Reg   

     LandContour Utilities  ... PoolArea PoolQC  Fence MiscFeature MiscVal  \
0            Lvl    AllPub  ...        0   None   None        None       0   
1            Lvl    AllPub  ...        0   None   None        None       0   
2            Lvl    AllPub  ...        0   None   None        None       0   
3            Lvl    AllPub  ...        0   None   None        None       0   
4            Lvl    AllPub  ...        0   None   None        None       0   
...          ...       ...  ...      ...    ...    ...         ...     ...   
1455         Lvl    AllPub  ...        0   None   None        None       0   
1456         Lvl    AllPub  ...        0   None  MnPrv        None       0   
1457         Lvl    AllPub  ...        0   None  GdPrv        Shed    2500   
1458         Lvl    AllPub  ...        0   None   None        None       0   
1459         Lvl    AllPub  ...        0   None   None        None       0   

     MoSold YrSold  SaleType  SaleCondition  SalePrice  
0         2   2008        WD         Normal     208500  
1         5   2007        WD         Normal     181500  
2         9   2008        WD         Normal     223500  
3         2   2006        WD        Abnorml     140000  
4        12   2008        WD         Normal     250000  
...     ...    ...       ...            ...        ...  
1455      8   2007        WD         Normal     175000  
1456      2   2010        WD         Normal     210000  
1457      5   2010        WD         Normal     266500  
1458      4   2010        WD         Normal     142125  
1459      6   2008        WD         Normal     147500  

[1460 rows x 81 columns]

In [6]:
#select numeric data and drop missing values
df = housing.frame.select_dtypes(['float', 'int']).dropna(axis = 1)#.select_dtypes(['int', 'float'])

In [10]:
df.shape

(1460, 35)

[Back to top](#Index:) 

## Problem 1

### Scale the data

**5 Points**

Scale the `df` data using its mean and standard deviation so that it is ready for SVD.  Assign the scaled data to `df_scaled` below.  

In [14]:
### GRADED

# YOUR CODE HERE
df_mu = df.mean()
df_sigma = df.std()
df_scaled = (df - df_mu) / df_sigma

# Answer check
print(type(df_scaled))
print(df_scaled.head())

<class 'pandas.core.frame.DataFrame'>
         Id  MSSubClass   LotArea  OverallQual  OverallCond  YearBuilt  \
0 -1.730272    0.073350 -0.207071     0.651256    -0.517023   1.050634   
1 -1.727900   -0.872264 -0.091855    -0.071812     2.178881   0.156680   
2 -1.725528    0.073350  0.073455     0.651256    -0.517023   0.984415   
3 -1.723156    0.309753 -0.096864     0.651256    -0.517023  -1.862993   
4 -1.720785    0.073350  0.375020     1.374324    -0.517023   0.951306   

   YearRemodAdd  BsmtFinSF1  BsmtFinSF2  BsmtUnfSF  ...  WoodDeckSF  \
0      0.878367    0.575228   -0.288554  -0.944267  ...   -0.751918   
1     -0.429430    1.171591   -0.288554  -0.641008  ...    1.625638   
2      0.829930    0.092875   -0.288554  -0.301540  ...   -0.751918   
3     -0.720051   -0.499103   -0.288554  -0.061648  ...   -0.751918   
4      0.733056    0.463410   -0.288554  -0.174805  ...    0.779930   

   OpenPorchSF  EnclosedPorch  3SsnPorch  ScreenPorch  PoolArea   MiscVal  \
0     0.21642

[Back to top](#Index:) 

## Problem 2

### Extracting $\Sigma$

**5 Points**

Using the scaled data, extract the singular values from the data using the `scipy.linalg` function `svd`.  Assign your results to `U`, `sigma`, and `VT` below. 

In [16]:
### GRADED

# YOUR CODE HERE
U, sigma, VT = svd(df_scaled, full_matrices=False)

# Answer check
print(type(sigma))
print(sigma.shape)

<class 'numpy.ndarray'>
(35,)


[Back to top](#Index:) 

## Problem 3

### Percent Variance Explained

**5 Points**

Divide `sigma` by the sum of the singular values to compute the percent variance explained. Assign your result as a percents array to `percent_variance_explained` below.  

Note that due to rounding this percent won't sum to exactly 1.  

In [18]:
### GRADED

sigma_sum = np.sum(sigma)
percent_variance_explained = sigma / sigma_sum

# YOUR CODE HERE

print(percent_variance_explained.shape)
print(percent_variance_explained.sum())

(35,)
1.0


[Back to top](#Index:) 

## Problem 4

### Cumulative Variance Explained

**5 Points**

Using the solution to problem 3, how many principal components are necessary to retain up to 80% of the explained variance if we consider them in descending order?  Assign your response to `ans4` below as an integer. 

**HINT**: explore the `np.cumsum` function.

In [22]:
### GRADED

# YOUR CODE HERE
ans4 = int((np.cumsum(percent_variance_explained) < .8).sum())
print(type(ans4))
print(ans4)

<class 'int'>
21
